In [1]:
%reload_ext autoreload
%autoreload 2

from utils import *

In [2]:
fpath = 'qa_corpus.csv'
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

content = pd.read_csv(fpath, encoding='utf-8')

In [3]:
# import jieba
import re


import os
LTP_DATA_DIR = 'D:/ProgramData/nlp_package/ltp_v34'  # ltp模型目录的路径
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`

from pyltp import Segmentor
segmentor = Segmentor()  # 初始化实例
segmentor.load_with_lexicon(cws_model_path, 'lexicon_seg.txt') # 加载外部词典文件路径


def token(string):
    return re.findall(r'[\d|\w]+', string)

def cut(string): 
    return ' '.join(segmentor.segment(string))

def filter_text(content):
    q_content = content['question'].tolist()
    q_content = [token(str(n)) for n in q_content]
    q_content = [' '.join(n) for n in q_content]
    q_content = [cut(n) for n in q_content]
    return q_content

In [4]:
q_content = filter_text(content)

In [5]:
stopwords = []
with open('chinese_stopwords.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        if len(line.strip()) < 2:
            stopwords.append(line.strip())

with open('哈工大停用词表.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        if len(line.strip()) < 2:
            stopwords.append(line.strip())

In [6]:
vectorized = TfidfVectorizer(max_features=12000, tokenizer=lambda x: x.split(), stop_words=stopwords)
X = vectorized.fit_transform(q_content)

In [7]:
import scipy.sparse as sp


X_array = X.toarray()
X_array = sp.csr_matrix(X_array)

In [8]:
import numpy as np


np.nonzero(X[100].toarray()[0])

(array([ 5278,  9413,  9499, 10423], dtype=int64),)

In [9]:
from scipy.spatial.distance import cosine

def distance(v1, v2): return cosine(v1, v2)

In [17]:
from operator import and_
from functools import reduce


pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')

from pyltp import Postagger
postagger = Postagger() # 初始化实例
postagger.load_with_lexicon(pos_model_path, 'lexicon.txt')  # 加载模型

and_pos_set = {'n', 'v', 'm', 'nh', 'ni', 'nl', 'ns', 'nt', 'ws'}


def token(string):
    return re.findall(r'[\d|\w]+', string)

def cut(string): 
    return segmentor.segment(string)

def filter_text_single(string):
    q_content = token(string.lower())
    print(q_content)
    q_content = ' '.join(q_content)
    q_content = cut(q_content)
    return q_content

In [23]:
word_2_id = vectorized.vocabulary_
id_2_word = {d: w for w, d in word_2_id.items()}

inverse_idx = X_array.transpose()

In [24]:
def search_connect_doc(query):
    """"""
    words = filter_text_single(query)
    postags = postagger.postag(words)

    to_and = []
    for i, postag in enumerate(postags):
        if postag in and_pos_set:
            to_and.append(words[i])
    print(to_and)
    
    query_vec = vectorized.transform([' '.join(words)]).toarray()
    try:
        candidates_ids = [word_2_id[w] for w in to_and]
    except KeyError:
        pass

    documents_ids = [
         set(np.nonzero(inverse_idx[_id].toarray()[0])[0]) for _id in candidates_ids
    ]

    merged_documents = reduce(and_, documents_ids)

    return merged_documents

In [25]:
search_connect_doc(content.question[1])

['代发工资']
['代发', '工资']


{1,
 24425,
 24821,
 24822,
 24823,
 24824,
 25316,
 25317,
 25320,
 25321,
 25748,
 25749,
 25799,
 26738,
 26739}

In [26]:
content.question[24425]

'代发工资客户个人信用消费贷款用途的使用范围'